In [2]:
!pip install datasets -q

In [3]:
!pip install deep_translator -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.5 MB/s eta 0:00:00


In [4]:
!pip install pandas openpyxl -q

In [1]:
from datasets import load_dataset

In [5]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
# dataset_train = load_dataset("/kaggle/input/image-processing-thai-language-image-captioning/train/train")
# dataset_val = load_dataset("/kaggle/input/image-processing-thai-language-image-captioning/val/val")

In [7]:
dataset_test = load_dataset("/kaggle/input/image-processing-thai-language-image-captioning/test/test")

Resolving data files:   0%|          | 0/2000 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
import numpy as np
import pandas as pd 
import os
from deep_translator import GoogleTranslator

In [9]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-23): 24 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (projection): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (layer_norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((1024,),

In [11]:
lst_image = []
test_dir = "/kaggle/input/image-processing-thai-language-image-captioning/test/test"

for dirname, _, filenames in os.walk(test_dir):
    for filename in filenames:
        image_path = os.path.join(dirname, filename)
        
        try:
            with Image.open(image_path) as img:
                img = img.convert("RGB")
                
            # Process input and move to GPU
            inputs = processor(img, return_tensors="pt").to(device)
            
            # Generate caption
            with torch.no_grad():
                output = model.generate(**inputs)
            
            # Decode caption
            caption = processor.decode(output[0], skip_special_tokens=True)
            
            # Translate to Thai
            thai_caption = GoogleTranslator(source="en", target="th").translate(caption)
            
            # Store result
            lst_image.append([filename ,thai_caption])
            print(f"✅ Processed: {filename} | Caption: {thai_caption}")

        except Exception as e:
            print(f"❌ Error processing {filename}: {e}")

print(f'Successfully task size:{len(lst_image)}')

✅ Processed: 00767.jpg | Caption: ภาพระยะใกล้ของแมลงขาวดำบนหิน
✅ Processed: 00266.jpg | Caption: มีป้ายเขียนภาษาไทยติดอยู่ด้านข้างอาคาร
✅ Processed: 01496.jpg | Caption: มีเห็ดจำนวนมากที่กำลังเติบโตบนพื้นดิน
✅ Processed: 01600.jpg | Caption: มีงูตัวหนึ่งนอนอยู่บนพื้น
✅ Processed: 00847.jpg | Caption: มีดอกไม้สีเหลืองที่กำลังเติบโตอยู่ในหญ้า
✅ Processed: 00822.jpg | Caption: มีรูปปั้นพระพุทธเจ้าและรูปเคารพทางศาสนามากมาย
✅ Processed: 00614.jpg | Caption: มีสัตว์สองตัวที่ยืนอยู่ในหญ้า
✅ Processed: 01850.jpg | Caption: มีต้นไม้เล็กๆ ต้นหนึ่งกำลังงอกออกมาจากพื้นดิน
✅ Processed: 00878.jpg | Caption: มีหนูตัวเล็ก ๆ ตัวหนึ่งกำลังนั่งอยู่บนพื้น
✅ Processed: 01473.jpg | Caption: รูปปั้นพระพุทธเจ้าสีทองอร่ามในห้องที่มีรูปปั้นอื่นๆ อีกมากมาย
✅ Processed: 00091.jpg | Caption: มีเจ้าหน้าที่ตำรวจ 3 นายยืนอยู่บนทางเดินไม้พร้อมป้ายสีเหลือง
✅ Processed: 01713.jpg | Caption: คนกำลังทำงานอยู่บนเตียงไม้ในห้องหนึ่ง
✅ Processed: 01306.jpg | Caption: มีของเล่นช้างสีส้มตัวเล็ก ๆ อยู่บนจานสีขาว
✅ Processed: 008

In [40]:
df = pd.DataFrame(lst_image, columns=["image_id", "caption"])

In [41]:
df.head(3)

,image_id,caption
0,00767.jpg,ภาพระยะใกล้ของแมลงขาวดำบนหิน
1,00266.jpg,มีป้ายเขียนภาษาไทยติดอยู่ด้านข้างอาคาร
2,01496.jpg,มีเห็ดจำนวนมากที่กำลังเติบโตบนพื้นดิน


In [42]:
df['image_id'] = df['image_id'].apply(lambda x: str(int(x.split('.')[0])))

In [43]:
sample_submission = pd.read_csv("/kaggle/input/image-processing-thai-language-image-captioning/sample_submission.csv")
sample_submission.head(3)

,image_id,caption
0,1354,ภาพถ่ายระยะใกล้ของวัตถุทรงกลมสีขาวที่มีคราบสีด...
1,1413,นกตัวหนึ่งที่กําลังเกาะอยู่บนกิ่งไม้อันหนึ่งที...
2,1802,บ้านที่อยู่ติดกับบริเวณริมชายหาดทะเลและมีต้นไม...


In [44]:
df['image_id'] = df['image_id'].astype(str)
sample_submission['image_id'] = sample_submission['image_id'].astype(str)

df_sorted = sample_submission[["image_id"]].merge(df, on="image_id", how="left")
df_sorted['image_id'] = df_sorted['image_id'].apply(lambda x: str(x).zfill(5))

In [45]:
df_sorted

,image_id,caption
0,01354,มีแมลงสีขาวตัวเล็ก ๆ อยู่บนทราย
1,01413,มีจิ้งจกตัวหนึ่งกำลังนั่งอยู่บนกิ่งไม้
2,01802,มีเรือลำหนึ่งแล่นผ่านหน้าผาหิน
3,01243,มีลิงตัวหนึ่งกำลังนั่งอยู่บนพื้นข้างกำแพง
4,00693,มีแมลงตัวเล็ก ๆ สองตัวอยู่บนพื้น
...,...,...
1995,00029,มีบ่อน้ำอยู่ในเรือนกระจกพร้อมต้นไม้และป้าย
1996,01065,มีดอกไม้สีแดงจำนวนมากบนต้นไม้ใบเขียว
1997,00195,ต้นไม้เบื้องหน้าภูเขาที่มีท้องฟ้าสีฟ้า
1998,01026,มีดอกไม้เล็ก ๆ ที่กำลังเติบโตบนลำต้น


In [46]:
df_sorted.to_excel('/kaggle/working/output1.xlsx', index=False, engine='openpyxl')

In [ ]:
# lst_image = []

# for dirname, _, filenames in os.walk(test_dir):
#     for filename in filenames:
#         image_path = os.path.join(dirname, filename)
#         image = Image.open(image_path).convert("RGB")
        
#         inputs = processor(image, return_tensors="pt").to(device)
        
#         with torch.no_grad():
#             output = model.generate(**inputs)

#         caption = processor.decode(output[0], skip_special_tokens=True)
#         thai_caption = GoogleTranslator(source='en', target='th').translate(caption)
        
#         lst_image.append(thai_caption)
#         print(f'Image_{filename} done!')

In [47]:
df_sorted.to_csv('/kaggle/working/submission.csv', index=False, encoding='utf-8-sig')

In [48]:
!gdown https://drive.google.com/uc?id=1DKre_hbW2IBhXkTmWFRAlhkRqz4EPMDJ

Downloading...
From: https://drive.google.com/uc?id=1DKre_hbW2IBhXkTmWFRAlhkRqz4EPMDJ
To: /kaggle/working/kaggle.json
100%|█████████████████████████████████████████| 67.0/67.0 [00:00<00:00, 312kB/s]


In [50]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [51]:
!kaggle competitions submit -c image-processing-thai-language-image-captioning -f submission.csv -m "Message"

100%|█████████████████████████████████████████| 231k/231k [00:00<00:00, 615kB/s]
Successfully submitted to Image Processing: Thai Language Image Captioning